# Import statements

In [ ]:
import os
import csv
import shutil
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta
import drms
import time

In [ ]:
# # Turn off tqdm
# from functools import partialmethod
# tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

# Define auxiliary functions

In [ ]:
## Utility Functions ##
def mkdir(a_dir):
    if(not os.path.exists(a_dir)):
        os.makedirs(a_dir)          
    
def write_csv(file_name, function_csv, data, header=False):
    with open(file_name, function_csv) as f:
        writer = csv.writer(f,lineterminator='\n')
        if header:
            writer.writerow(header)
        writer.writerow(data)    
        
def read_csv(csv_path):
    with open(csv_path, newline='') as f:
        reader = csv.reader(f)
        data = list(reader)    
    return data        

def write_txt(txt_path, function_txt_downloaded_flares, flare_id, header=False):
    if function_txt_downloaded_flares=='w':
        with open(txt_path, function_txt_downloaded_flares) as f:
            f.write(header)
            f.write('\n')
            f.write(flare_id)
            f.write('\n')
    else:
        with open(txt_path, function_txt_downloaded_flares) as f:
            f.write(flare_id)
            f.write('\n')

def read_txt(txt_path):
    lines = []
    with open(txt_path) as f:
        for line in f:
            lines.append(line.replace("\r", "").replace("\n", ""))
    return lines
            
            
def convert_time_format(cur_time, offset=0):
    # offset: current - or + certain minutes
    #         e.g., start_time - 15: offset=-15
    temp = datetime.strptime(cur_time, '%Y-%m-%d %H:%M:%S') + timedelta(minutes = offset)
    temp = temp.strftime("%Y-%m-%dT%H:%M:%S")
    return temp

        
def write_csv_event(file_name, flare_id, goes_class, noaa_active_region, primary_verified, 
                    secondary_verified, x_hpc, y_hpc, start_time, end_time, peak_time, 
                    tref, delta_t, n_pixel):
    
    header_csv = ['flare_id', 'goes_class', 'noaa_active_region', 'primary_verified', 
                  'secondary_verified', 'x_hpc', 'y_hpc', 'start_time_flare', 'end_time_flare', 
                  'peak_time_flare', 'tref', 'delta_t', 'n_pixel']

    data = [flare_id, goes_class, noaa_active_region, primary_verified, 
            secondary_verified, x_hpc, y_hpc, start_time, end_time, peak_time, 
            tref, delta_t, n_pixel]

    function_csv = 'w'

    with open(file_name, function_csv, encoding='UTF8', newline='') as file_csv:

            writer = csv.writer(file_csv)
            writer.writerow(header_csv)
            writer.writerow(data)

# download aia images through drms
def download_aia_images(tref, delta_t, x_hpc, y_hpc, n_pixel,
                        path_dir='./', notify='john_doe@something.com'):
    
    client = drms.Client(verbose=False)
    
    qstr = "aia.lev1_euv_12s["+ tref +"/" + str(delta_t) + "m]"
    print(f"Data export query:\n  {qstr}\n")

    ###############################################################################
    # Construct the dictionary specifying that we want to request a cutout.
    # This is done via the ``im_patch`` command.
    
    # The ``t`` controls whether tracking is disabled (``1``) or enabled (``0``).
    # ``r`` controls the use of sub-pixel registration.
    # ``c`` controls whether off-limb pixels are filled with NaNs.
    # For additional details about ``im_patch``, 
    # see the `documentation <http://jsoc.stanford.edu/doxygen_html/group__im__patch.html>`_.
    process = {
        "aia_scale_aialev1":{           # Scale for transforming data as level 1p5
        "mpt": "aia.master_pointing3h",
        },
        "im_patch": {
            "t_ref": tref,
            "t": 1,
            "r": 0,
            "c": 0,
            "locunits": "arcsec",
            "boxunits": "pixels",
            "x": x_hpc,
            "y": y_hpc,
            "width": n_pixel,
            "height": n_pixel,
        }
    }

    
    # Submit export request using the 'fits' protocol
    print("Submitting export request...")
    result = client.export(
        qstr,
        method="url",
        protocol="fits",
        email=notify,
        process=process,
        filenamefmt="{seriesname}.{T_REC:A}.{WAVELNTH}.{segment}",
    )

    # Print request URL.
    print(f"\nRequest URL: {result.request_url}")
    print(f"{int(len(result.urls))} file(s) available for download.\n")

    # Download selected files.
    result.wait()
    result.download(path_dir)
    print("Download finished.")

# Define main function for downloading AIA data

In [ ]:
def download_single_event(flare_id, goes_class, noaa_active_region, primary_verified, secondary_verified, 
                          x_hpc, y_hpc, start_time, end_time, peak_time, tref, 
                          delta_t, n_pixel, aia_fits_folder, notify, downloaded_flares):
    
                          

    print('*********\nStart time:', datetime.now())
    print('Working on flare: "%s"' %flare_id)
    
    ## Check if the flare is downloaded ##
    if(flare_id in downloaded_flares):
        print('The flare has been downloaded before.')
        print('Completed!')
        print('End time:', datetime.now())
        s = '[Error Code: -1] The flare has been downloaded before'
        return -1, s, flare_id
      
    print('Dowloading AIA data...', datetime.now())
    
    ## Create folder for this event
    event_folder = os.path.join(aia_fits_folder,flare_id)
    mkdir(event_folder)
    
    download_aia_images(tref, delta_t, x_hpc, y_hpc, n_pixel, path_dir=event_folder, notify=notify)

    
    file_name = os.path.join(event_folder, flare_id + "_data.csv")
    
    write_csv_event(file_name, flare_id, goes_class, noaa_active_region, primary_verified, 
                    secondary_verified, x_hpc, y_hpc, start_time, end_time, peak_time, 
                    tref, delta_t, n_pixel)
        
    print('End time:', datetime.now())
    print()
    
    return 0, '', flare_id 

# Set parameters

In [ ]:
project_dir = "."

data_dir     = os.path.join(project_dir, "data") # parent dir for all data folders
flare_lists_dir = os.path.join(data_dir, "flare_lists")

aia_fits_folder = os.path.join(data_dir, "aia_fits")
if not os.path.exists(aia_fits_folder):
    os.mkdir(aia_fits_folder)

# Setup Hyperparameters
delta_t = 30 # in minutes
n_pixel = 1000


downloaded_flares_file = 'downloaded_flares.txt'
notify = 'paolocrmassa@gmail.com' # please enter a valid email address for Jsoc
flare_list = os.path.join(flare_lists_dir, "flare_list_1.csv")

# Download data

In [ ]:
idx = flare_list.find('flare_list_')
flare_list_name = flare_list[idx:-4]

flare_list_folder = os.path.join(aia_fits_folder, flare_list_name)

# Create folder for this flare list
if not os.path.exists(flare_list_folder):
    os.mkdir(flare_list_folder)

# Create metadata dir
metadata_dir = os.path.join(flare_list_folder, "metadata")
if not os.path.exists(metadata_dir):
    os.mkdir(metadata_dir)

# Load the list of downloaded flares
# If such file does not exist, create an empty one
downloaded_flares_path = os.path.join(metadata_dir, downloaded_flares_file)
if(os.path.exists(downloaded_flares_path)):
    downloaded_flares = read_txt(downloaded_flares_path)
    function_txt_downloaded_flares = "a"
else:
    downloaded_flares = []
    function_txt_downloaded_flares = "w"

# Load previous log
# If such file does not exist, create an empty one
error_log_file_path = os.path.join(metadata_dir, 'error_log.csv')
if(os.path.exists(error_log_file_path)):
    function_csv_error_log = "a"
else:
    function_csv_error_log = "w"

# Load the list of all flares that need to be downloaded
flares = np.asarray(read_csv(flare_list)[1:])

#for i in tqdm(range(len(flares)), disable=True):
for i in range(len(flares):

    flare_id, goes_class, noaa_active_region, primary_verified, secondary_verified, x_hpc, y_hpc, start_time, end_time, peak_time = flares[i]

    # flare start and end times
    flare_start = convert_time_format(start_time, 0)
    flare_end   = convert_time_format(end_time, 0)
    flare_peak  = convert_time_format(peak_time, 0)

    # tref: start time of the data to be downloaded
    tref   = convert_time_format(start_time, -delta_t)

    try:
        # download a flare
        error_code, result, flare_id = download_single_event(flare_id, goes_class, noaa_active_region, primary_verified, secondary_verified, 
                                                             x_hpc, y_hpc, flare_start, flare_end, flare_peak, tref, 
                                                             delta_t, n_pixel, flare_list_folder, notify, downloaded_flares)

        downloaded_flares.append(flare_id)

        if error_code!=-1:
            # write downladed flares to file
            if function_txt_downloaded_flares=="w":
                write_txt(downloaded_flares_path, function_txt_downloaded_flares, flare_id, header='flare_id')
                function_txt_downloaded_flares="a"
            else:
                write_txt(downloaded_flares_path, function_txt_downloaded_flares, flare_id)

    except Exception as e:
        result = str(e)
        if function_csv_error_log=="w":
            write_csv(error_log_file_path, function_csv_error_log, [flare_id, result], header=['flare_id','error'])
            function_csv_error_log="a"
        else:
            write_csv(error_log_file_path, function_csv_error_log, [flare_id, result])
        pass
    
    print("")
    print("")
    print("Sleep for 30s before the next request")
    time.sleep(30)
    print("")
    print("")